In [11]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_tensor_type(torch.DoubleTensor)

import matplotlib.pyplot as plt 

if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu"  


### 1. Gradient descent for the finite neuron method: $L^2$-fitting

$$\sum_{i=1}^n a_i \sigma(\omega_i x + b_i)$$

1.1 Neural network architecture 

1.2 Access and modify parameters in a neural network 

1.3 Define a loss function
    	\begin{equation} \label{snn-optimization}
			 \min_{a_i, b_i} \int_{-\pi}^{\pi} \frac{1}{2} |f(x) - \sum_{i=1}^{n} a_i \sigma( x + b_i) |^2 dx.  
		\end{equation}
        
1.4 Create an optimizer in pytorch and tune it parameters 



In [27]:
## 1.1 Neural network architecture 
class model(nn.Module):
    def __init__(self, input_size, hidden_size1, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) # W1, b
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False) # W2

    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x)))
        return u1


In [ ]:
## 1.2 Access and modify parameters in a neural network 
"""
Instantiate a class object my_model
Access the parameters
Mutate(modify) the parameters
"""
hidden_layer = 4
my_model = model(1,hidden_layer,1)

"""
1. Access parameters using my_model.parameters()
2. torch parameters contain two attributes: data and requires_grad
"""
for parameter in my_model.parameters():
    print(parameter.data)
    print(parameter.requires_grad)
print()

"""
1. Directly access the class attributes: fc1, fc2
2. Access the paramters as attributes of fc1, fc2
"""
print(my_model.fc1.weight)
print(my_model.fc1.bias)
print(my_model.fc1.weight.data)
print(my_model.fc1.weight.requires_grad)
print()
print("+++++++++++++")
for param in my_model.fc1.parameters():
    print(param)
    param.requires_grad = False 
    param.requires_grad_()
print(my_model.fc1.weight)
print(my_model.fc1.bias)

"""
Modify the parameter:
Shape matches; modify data and requires_grad separately
"""
print("=========")
w_i = torch.full((1,hidden_layer),1.0).view(hidden_layer,1)
print(my_model.fc1.weight.data)
my_model.fc1.weight.data = w_i # w_i is now the data
my_model.fc1.weight.requires_grad = False 
print(my_model.fc1.weight)
print(my_model.fc1.weight.data)
print(my_model.fc1.weight.requires_grad)

In [6]:
## 1.3 Define a loss function using piecewise Gauss quadrature

#### Standard p-point Gaussian quadrature rule on $[-1,1]$
\begin{equation}
    \int_{-1}^{1} f(x) dx \approx \sum_{i = 1}^p w_i f(g_i).
\end{equation}

On an arbitrary interval $[x_i, x_{i+1}]$: 
\begin{equation}
\begin{aligned}
        \int_{x_i}^{x_{i+1}} f(x) dx & = \frac{x_{i+1} - x_{i}}{2 } \int_{-1}^1 f(\frac{x_{i+1} - x_i}{2} \xi + \frac{x_{i+1} + x_i}{2}) d\xi \\
        & \approx \frac{x_{i+1} - x_{i}}{2 } \sum_{j =1}^p w_j f(\frac{x_{i+1} - x_i}{2}  g_j + \frac{x_{i+1} + x_i}{2}) 
\end{aligned}
\end{equation}

**Method**
- Divide the domain (interval) into several subdomains (subintervals). $[x_0,x_1], [x_1,x_2],...,[x_{N-1}, x_{N}]$
- Compute the quadrature value in each subdomain.
- Sum the quadrature values in each subdomain to get the quadrature value over the whole domain.
- vectorization 

**Remark**
- Implementing a monte carlo or trapezoidal rule is eaiser but not as accurate.

##### Speed up by vectorization? 

1. on each subinterval $[x_i, x_{i+1}]$
    - $\frac{x_{i+1} - x_{i}}{2 } \sum_{j =1}^p w_j f(\frac{x_{i+1} - x_i}{2}  g_j + \frac{x_{i+1} + x_i}{2}) $


In [32]:
### 5 point Gauss Quadrature rule

def integrand(x):
    return torch.sin(10*x)+1

gx = torch.tensor([-0.9061798459386639927976, -0.5384693101056830910363, 0, 0.5384693101056830910363, 
     0.9061798459386639927976]).to(device)
gx = gx.view(1,-1)
gw = torch.tensor([0.2369268850561890875143, 0.4786286704993664680413, 0.5688888888888888888889, 0.4786286704993664680413,
     0.2369268850561890875143]).to(device)
gw = gw.view(-1,1) # Column vector 
num_points = 1 # subintervals
nodes = torch.linspace(-pi,pi,num_points+1).view(-1,1) 
coef1 = ((nodes[1:,:] - nodes[:-1,:])/2) # n by 1  
coef2 = ((nodes[1:,:] + nodes[:-1,:])/2) # n by 1  
coef2_expand = coef2.expand(-1,gx.size(1)) # Expand to n by p shape, -1: keep the first dimension n , expand the 2nd dim (columns)
integration_points = coef1@gx + coef2_expand
integration_points = integration_points.flatten().view(-1,1) # Make it a column vector
gw_expand = torch.tile(gw,(num_points,1)) # rows: n copies of current tensor, columns: 1 copy, no change
# Modify coef1 to be compatible with func_values
coef1_expand = coef1.expand(coef1.size(0),gx.size(1))    
coef1_expand = coef1_expand.flatten().view(-1,1)

func_values = integrand(integration_points)

integral_value = torch.sum(func_values*gw_expand*coef1_expand)
print(integral_value)


tensor(5.6612)


##### 1.4 Create an optimizer in pytorch and tune it parameters 
- import torch.optim as optim
- link: https://pytorch.org/docs/stable/optim.html
- Schedule learning rate

- Basic syntax:

        optimizer.zero_grad() # clear the gradient wrt parameters 
    
        output = model(input) 
    
        loss = loss_fn(output, target) # define the loss 
    
        loss.backward() # compute the gradients 
    
        optimizer.step() # gradient step: $\theta^{k+1} = \theta^k - \nabla_\theta L(\theta^k)$

In [ ]:
## 1.4 Create an optimizer in pytorch and tune it parameters 

# Define optimizer 
optimizer1 = optim.SGD(my_model.parameters(), lr=0.02)

# Learning rate schedule
lr = 0.02
for param_group in optimizer1.param_groups:
    param_group['lr'] = lr * (0.98 ** ((epoch + 1) // 1000)) # Learning rate schedule
    

## Orthogonal greedy algorithm for finite neuron method: $L^2$-fitting

**How to implement the two major steps in OGA?**

1. Solve the argmax problem $\quad g_n  = \mathop{\arg \max}_{g \in \mathbb{D}}  | \langle g, f - f_{n-1}
			\rangle | $ 
    - $L^2$ innner product: numerical integration 
        - Piecewise Gauss quadrature (very accurate numerical quadrature is needed for this part)
    - **Method of exhaustion** for 1D for a good initial guess 
        - $\mathbb{D}_N = \{ \sigma(x + b_i), b_i = \pi -  2 \pi\frac{i-1}{N} \}_{i = 1}^N$
        - $\quad g_n  = \mathop{\arg \max}_{g \in \mathbb{D_N}}  | \langle g, f - f_{n-1}
			\rangle | $ 
            - store values of $g$ as a row vector, values of $f - f_{n-1}$ column vector
    - Further optimization using Newton's method to get a better $g_n$ (This part is also essential when the number of neuron is large)
    

In [41]:
## 1. ReLU dictionary discretization 
# relu dictionary
def relu_dict(x_l,x_r,N):

    relu_dict_parameters = torch.zeros((N,2))
    relu_dict_parameters[:N,0] = torch.ones(N)[:]
    relu_dict_parameters[:N,1] = torch.linspace(x_l,x_r,N+1)[1:] # relu(x+bi)

    return relu_dict_parameters
def target(x):
    return x 
relu_dict_parameters = relu_dict(-pi,pi,100)
integration_intervals = 10 
nodes = torch.linspace(-pi,pi,integration_intervals+1).view(-1,1) 
coef1 = ((nodes[1:,:] - nodes[:-1,:])/2) # n by 1  
coef2 = ((nodes[1:,:] + nodes[:-1,:])/2) # n by 1  
coef2_expand = coef2.expand(-1,gx.size(1)) # Expand to n by p shape, -1: keep the first dimension n , expand the 2nd dim (columns)
integration_points = coef1@gx + coef2_expand
integration_points = integration_points.flatten().view(-1,1) # Make it a column vector
gw_expand = torch.tile(gw,(integration_intervals,1)) # rows: n copies of current tensor, columns: 1 copy, no change
coef1_expand = coef1.expand(coef1.size(0),gx.size(1))    
coef1_expand = coef1_expand.flatten().view(-1,1)
func_values = target(integration_points) 

weight_func_values = func_values*gw_expand*coef1_expand
print(weight_func_values.size())
basis_values = F.relu(relu_dict_parameters[:,0] *integration_points + relu_dict_parameters[:,1]).T # uses broadcasting  
print(basis_values.size())
output = torch.abs(torch.matmul(basis_values,weight_func_values)) # 
print(output.size())
neuron_index = torch.argmax(output.flatten())

print(neuron_index)


torch.Size([50, 1])
torch.Size([100, 50])
torch.Size([100, 1])
tensor(99)


2. Orthogonal projection. Given $H_n = \text{span}\{g_1, g_2,...,g_n \}$
    - Use w_list, b_list to keep track of the added neurons
        - reconstruct the neural network from the two lists 
    - Solve the linear system: 
         - $<f_n,g_i> = <f,g_i>$ for all $i= 1,2,...,n$, with $f_n = \sum_{i=1}^n a_i g_i $
         - A liner system in $\alpha := (a_1, a_2,...,a_n)^T$. $M \alpha = b$, where $M_{i,j} = <g_j,g_i>$ $b_i = <f,g_i> $
         - Equivalently, an $L^2$-fitting: $\min_{a_1, a_2,...,a_n} \frac{1}{2} \|\sum_{i=1}^n a_i g_i  - f \|^2_{L^2} $, that is $\min_{\alpha \in \mathbb{R}^n} \frac{1}{2} \alpha^T M \alpha - b^T \alpha $
    - A shortcut: make use of pytorch automatic differentiation to get the matrix M, since we already know how to compute the loss function very accurately.


#### Extract the linear system using pytorch auto differentiation and solve the linear system 

$L(x) = \frac{1}{2}x^T A x$

$\nabla_x L(x) = A x$

$\nabla_x (Ax)_i = \text{ ith row of } A $


In [ ]:
def minimize_linear_layer(model,target,solver="cg"):
    """Solve the linear layer problem Mx = b: L2 fitting relu shallow neural networks 
    Parameters
    ----------
    model : 
        relu shallow neural network
    target: 
        a target function 
        
    Returns
    -------
    sol: tensor 
        the solution of the linear system 
    """
    def loss_function_inside(x):
        return 0.5*torch.pow(model(x)-target(x),2).to(device)

    def rhs_loss_inside(x): 
        return model(x)*target(x)
    #1. Compute loss function using piecewise Gaussian quadrature
    node = compute_integration_nodes_relunn(-pi,pi,model)
    loss = GQ_piecewise(gw,gx,node,loss_function_inside) #loss_function

    #2. Extract the linear system A using torch.autograd 
    du1 = torch.autograd.grad(outputs=loss, inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
    jac = '' 
    neuron_number = model.fc1.bias.size(0)
    for i in range(neuron_number): 
        duui = torch.autograd.grad(outputs=du1[0,i], inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
        if i == 0: 
            jac = duui
        else: 
            jac = torch.cat([jac,duui],dim=0)

    #3. Extract the right hand side
    loss_helper = GQ_piecewise(gw,gx,node,rhs_loss_inside)
    rhs = torch.autograd.grad(outputs=loss_helper, inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
    rhs = rhs.view(-1,1)

    #4. Solve the linear system 
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach()),np.array(rhs.detach()),tol=1e-10) 
    elif solver == "direct": 
        sol = np.linalg.inv( np.array(jac.detach()) )@np.array(rhs.detach())
    sol = torch.tensor(sol).view(1,-1)
    return sol 
